# StreamVC Training on Google Colab

このノートブックでは、Google Colab上でStreamVCの学習を実行します。

## 前提条件
- Google Driveに`streamVC/data/`がアップロード済み
- GitHubリポジトリが公開されている

## 1. GPU確認

In [ ]:
!nvidia-smi

## 2. Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. リポジトリのクローン

In [ ]:
# GitHubリポジトリのURL（あなたのリポジトリに変更してください）
REPO_URL = "https://github.com/YOUR_USERNAME/streamVC.git"

!git clone {REPO_URL}
%cd streamVC

## 4. 依存パッケージのインストール

In [ ]:
!pip install -e . -q

## 5. Google Driveからデータセットをリンク

In [ ]:
import os

# Google Drive上のデータパス
GDRIVE_DATA = "/content/drive/MyDrive/streamVC/data"

# シンボリックリンクを作成
if os.path.exists("data"):
    !rm -rf data

!ln -s {GDRIVE_DATA} data

# 確認
!ls -lh data/

## 6. Checkpointディレクトリの設定

In [ ]:
# Checkpoint保存先をGoogle Driveに設定
CHECKPOINT_DIR = "/content/drive/MyDrive/streamVC/checkpoints"

!mkdir -p {CHECKPOINT_DIR}

print(f"Checkpoints will be saved to: {CHECKPOINT_DIR}")

## 7. 設定ファイルの準備（Colab用）

In [ ]:
# Colab用の設定ファイルを確認
!cat configs/colab_gpu_training.yaml

## 8. 学習の実行

### オプション1: インタラクティブ実行（ノートブック内で実行）

In [ ]:
# 学習を開始（Ctrl+Cで中断可能）
!python scripts/train.py \
    --config configs/colab_gpu_training.yaml \
    --device cuda \
    --gdrive-backup {CHECKPOINT_DIR}

### オプション2: バックグラウンド実行

In [ ]:
# バックグラウンドで学習を実行
!nohup python scripts/train.py \
    --config configs/colab_gpu_training.yaml \
    --device cuda \
    --gdrive-backup {CHECKPOINT_DIR} \
    > train.log 2>&1 &

print("Training started in background")
print("Check progress: !tail -20 train.log")

## 9. 学習の監視

In [ ]:
# ログの確認
!tail -30 train.log

In [ ]:
# TensorBoard起動
%load_ext tensorboard
%tensorboard --logdir {CHECKPOINT_DIR}/logs

## 10. Checkpointの確認

In [ ]:
# 保存されたCheckpointを確認
!ls -lh {CHECKPOINT_DIR}/

## 11. 学習の再開（セッション切断後）

In [ ]:
# 最新のCheckpointから再開
import glob

checkpoints = sorted(glob.glob(f"{CHECKPOINT_DIR}/step_*.pt"))
if checkpoints:
    latest_ckpt = checkpoints[-1]
    print(f"Resuming from: {latest_ckpt}")
    
    !python scripts/train.py \
        --config configs/colab_gpu_training.yaml \
        --device cuda \
        --gdrive-backup {CHECKPOINT_DIR} \
        --resume {latest_ckpt}
else:
    print("No checkpoint found. Starting from scratch.")

## Tips

### セッション切断対策
- Colabは最大12時間（無料版）または24時間（Pro）で切断されます
- Checkpointは自動的にGoogle Driveに保存されるので安全です
- 再接続後、「11. 学習の再開」から続行できます

### メモリ不足の場合
```yaml
# configs/colab_gpu_training.yaml
training:
  batch_size: 8  # 16 → 8に減らす
```

### データセットの追加
1. ローカルで新しいデータセットをダウンロード
2. `./scripts/upload_dataset_to_gdrive.sh`で追加アップロード
3. Configファイルに追加して再学習